In [1]:
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd
import json

In [2]:
variable = ['has_residential', 'has_market_rate', 'has_affordable_lowinc', 'has_livework', 'has_adu', 'has_mixeduse']
fine_tuned_model = "ft:gpt-4o-mini-2024-07-18:personal:housing-desc-trainset100-multirun3:BGxekKBX"

directory = "data"
data_file = "housing_descriptions_training.csv"
data_path = directory + "/" + data_file

data_df = pd.read_csv(data_path)
data_df.head()


,short_description,entitlement,proposed_adding,residential_add,adu_udu_add,multi_family_add,single_family_add,has_residential,has_market_rate,has_affordable_lowinc,has_livework,has_adu,has_udu,has_adu_udu,has_multi_family,has_single_family,has_non_res_sqft,has_mixeduse
0,NaN,ADDITIONAL GRADING IN COMPLIANCE WITH AND TO A...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NaN,DELETE CONDITION S-3(I)(A) OF VTT 71898,49.0,49.0,0.0,49.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,NaN,DEMOLITION OF EXISTING BUILDING TO CREATE TWO ...,75.0,75.0,0.0,75.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,NaN,"INCREASE GRADING, 2 NEW RETAINING WALLS TO COR...",1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NaN,MODIFICATION FROM AN APPROVED 12 UNIT DENSITY ...,12.0,12.0,0.0,12.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [3]:
# Sample rows for training set
train_df = data_df.sample(n=70, random_state=42)

# Sample additional rows for test set, ensuring no overlap with train set
test_df = data_df[~data_df.index.isin(train_df.index)].sample(n=100, random_state=8)

# Extract only the columns we need for both sets
# Using the variable list defined at the top of the file
columns_to_extract = ['short_description', 'entitlement'] + variable
train_df = train_df[columns_to_extract]
test_df = test_df[columns_to_extract]

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# Display first few rows of each set
print("\nTraining set sample:")
train_df.head()

Training set shape: (70, 8)
Test set shape: (100, 8)

Training set sample:


,short_description,entitlement,has_residential,has_market_rate,has_affordable_lowinc,has_livework,has_adu,has_mixeduse
1128,"CONSTRUCTION OF A NEW 2,838 SQFT SINGLE FAMILY...",PURSUANT TO LAMC SECTION 11.5.7(C) THE APPLICA...,1.0,1.0,0.0,0.0,0.0,0.0
4109,TRACT FOR 5 SMALL LOT SUBDIVISION CASE.,"PURSUANT TO LAMC 17.03, A REQUEST FOR A VESTIN...",1.0,1.0,0.0,0.0,0.0,0.0
3538,PROPOSED SMALL LOT SUBDIVISION TO CREATE 18 SI...,"PURSUANT TO LAMC 17.15, A VESTING TENTATIVE TR...",1.0,1.0,0.0,0.0,0.0,0.0
3613,"PURSUANT TO LAMC CODE SECTION 14.3.1, PROPOSED...","PURSUANT TO LAMC CODE SECTION 14.3.1, PROPOSED...",1.0,1.0,0.0,0.0,0.0,0.0
1861,DEMOLITION OF (E) MULTI-FAMILY STRUCTURE AND C...,PURSUANT TO LAMC SECTION 12.20 TO REQUEST A CO...,1.0,1.0,0.0,0.0,0.0,0.0


In [4]:
instructions = """
You are a housing project assistant. Read the **Entitlement** and **Short Description** fields. Your job is to classify whether each project has the following traits:

## VARIABLES
- **Res**: `1` if the project includes any **residential use**; `0` otherwise  
- **Mkt**: `1` if the project includes or implies any **market-rate housing**; `0` otherwise  
- **Aff**: `1` if the project includes any **LOWINC** (affordable or income-restricted) housing; `0` otherwise  
- **Liv**: `1` if the project includes any **live/work units** (residential unless explicitly commercial); `0` otherwise  
- **Adu**: `1` if the project includes any **Accessory Dwelling Units (ADUs)** or **Junior ADUs (JADUs)**; `0` otherwise  
- **Mix**: `1` if the project is **mixed-use** (includes both residential and non-residential uses); `0` otherwise


---

### Shared Definitions:


**LOWINC** refers to explicitly income-restricted housing, such as:
- Affordable housing
- Low-income, very low income, extremely low income
- Moderate income
- Supportive housing
- Income-restricted units
- Units required by a Density Bonus, Mello Act, or other affordability programs


**RES USES** (residential uses) include:
- Apartments, condos, dwelling units, single-family homes (SFD), ADUs, JADUs, duplexes, triplexes, fourplexes, live/work units (if no separate commercial), small-lot subdivisions

**Non-Residential Uses** include:
- Commercial, retail, restaurant, bar, office, administrative office, medical, industrial, warehouse
- Institutional uses: community center, school, religious facility

**Mixed-Use** means a project includes **both** residential and non-residential uses (see above)

**Live/Work Units** are considered **residential** unless explicitly stated that they include **separate or public-facing commercial space**


---

## VARIABLE: `Res`


### Task:
Determine whether the project includes **any residential use**.


### Output **1** if there is residential:
- Any of the following appear: apartments, housing units, condos, homes, duplex, triplex, fourplex, ADU, JADU, live/work units (unless explicitly non-residential)
- Even if only one residential unit is included (e.g., "1 apartment above a retail store")

### Output **0** if there is no residential:
- No residential use is mentioned
- The project only includes commercial, office, industrial, or institutional use without any housing

**Common Misleading Phrases**:
- "FAR", "Density Bonus", "Height Increase", and "Open Space Waiver" do **not** imply mixed-use by themselves.
- Garages, basements, storage, and parking are **not** considered non-residential uses.

---

## VARIABLE: `Mkt`


### Task:

### Output **0** if:
- All new or all existing **RES USES** are explicitly LOWINC, , or if the project involves **no net change** to housing quantity or type
- The only action is **legalizing** an already existing, previously unpermitted *dwelling unit*, with **no new structure or change in use**.

### Output **1** if:
- Output **1** if **any** of the following are true: new units are created or enabled, affordability is partial or missing, or units are altered in a way that increases market value.
- Any **new unit** is created: including ADUs, duplexes, or conversion of non-housing space (e.g., garage, rec room, basement) into a dwelling.
- A **unit is rebuilt**, enlarged, or significantly upgraded (e.g., demo and rebuild of SFD, or manufactured home replacement).
- An **existing unit is converted into another use** (e.g., to condos, bed & breakfast, or short-term rental).
- A **density bonus** or other incentive is used and *not all units* are LOWINC.
- A **zone change**, tract map, subdivision, or other **land entitlement** is requested and **no affordability is mentioned**.
- **Affordability is partial** (e.g., “2 of 26 units are affordable”) — remaining units are assumed market-rate.
- The language indicates **intent to redevelop, intensify, or commercialize housing** (e.g., demolition, change of use, upzoning).

### Assumptions:
- Assume **market-rate intent** unless it is clearly stated that **100% of units are LOWINC**.
- **Adding, enlarging, converting, or replacing** units implies market-rate housing unless explicitly affordable.

---

## VARIABLE: `Aff`


### Output:

- **1** if the project includes any units that are explicitly income-restricted or affordable
- **0** if there is no mention of any income-restricted, affordable, or low-income housing


### Criteria:

#### Output **1** if:
- if the project includes any units that are explicitly **LOWINC** (income-restricted or affordable)

#### Output **0** if:
- Residential units **RES USES** are included, but there is **no mention of affordability** or income restriction
- The **RES USES** are market-rate by implication (e.g., new SFDs, ADUs, apartments, condos without affordability language)
- The only activity is legalization or change-of-use (e.g., “legalize unpermitted unit”)


---

## VARIABLE: `Liv`


### Output:


### Criteria:

#### Output **1** if:
- if the project explicitly mentions live/work units
- The text contains **explicit phrases**, such as:
  - “live/work unit(s)”
  - “live-work space”
  - “live/work loft(s)”
  - “living/working quarters”

#### Output **0** if:
- otherwise
- There is **no explicit mention** of live/work units, even if the project is mixed-use, commercial-residential, or includes flexible space.


---

## VARIABLE: `Adu`

### Output **1** if:
- if the project includes an ADU
- The text explicitly mentions:
  - “ADU”
  - “Accessory Dwelling Unit”
  - “JADU” (Junior ADU)
- OR if there is strong indirect evidence:
  - “Garage conversion” (only if a unit is being added or repurposed for living space)
  - “Recreation room” or “Rec room” being added, legalized, or modified with a garage
  - “Second-story addition” to a garage or detached structure
  - “Non-conforming addition” with use or structure implying habitable space
  - Zoning language around “Accessory use”, “reduced side yard” + garage additions in residential zones
Only infer ADUs from **indirect language** when it strongly suggests a residential or second unit being added or legalized.

### Output **0** if:
- Otherwise
- There is no mention of “ADU”, “Accessory Dwelling Unit”, or “JADU”
- The project involves a garage, recreation room, or addition, but **not** as living space or housing
- The text discusses subdivision, zoning, or construction of new homes **without** referencing an ADU


---


## VARIABLE: `Mix`


### Task:
Determine whether the project is **mixed-use**, meaning it includes both **RES USE** and **Non-Residential Uses** uses.


**Mixed-Use** means a project includes **both** residential and non-residential uses (see above)
**Live/Work Units** are considered **residential** unless explicitly stated that they include **separate or public-facing commercial space**

**Common Misleading Phrases**:
- "FAR", "Density Bonus", "Height Increase", and "Open Space Waiver" do **not** imply mixed-use by themselves.
- Garages, basements, storage, and parking are **not** considered non-residential uses.

### Output **1** if:
- Output **1** if the project includes both residential **RES USE** and non-residential **Non-Residential Uses** components (or explicitly says "mixed-use")
- Residential **RES USE** and non-residential uses are both present

### Output **0** if:
- Otherwise
- The project is only residential (even if it has garages, basements, or parking)
- The project is only non-residential **Non-Residential Uses**
- The project includes live/work units, but no separate or public-facing commercial space is described



---

### Example:

Short Description:
"A DENSITY BONUS TO ALLOW A 5-STORY, 70-UNIT SENIOR HOUSING DEVELOPMENT, INCLUDING AFFORDABLE AND MARKET-RATE UNITS."

Entitlement:
"17 units set aside as low-income; remaining 53 units are market-rate."

**Expected Output Format:**
```json
{
  "Res": 1,
  "Mkt": 1,
  "Aff": 1,
  "Liv": 0,
  "Adu": 0,
  "Mix": 1
}

"""

In [5]:

# Prepare the data for fine-tuning
def prepare_finetune_data(df):
    examples = []
    
    for _, row in df.iterrows():
        # Get the text input (short_description and entitlement)
        short_desc = str(row['short_description']) if not pd.isna(row['short_description']) else ""
        entitlement = str(row['entitlement']) if not pd.isna(row['entitlement']) else ""
        
        # Combine the text inputs
        text = f"Short Description: {short_desc}\nEntitlement: {entitlement}"
        
        # Get the label (proposed_adding)
        label = row[variable]
        
        # Create the example in the required format
        example = {
            "messages": [
                {"role": "system", "content": instructions},
                {"role": "user", "content": text},
                {"role": "assistant", "content": str(label)}
            ]
        }
        
        examples.append(example)
    
    return examples

# Prepare training and test data
train_examples = prepare_finetune_data(train_df)
test_examples = prepare_finetune_data(test_df)

# Create directory if it doesn't exist
os.makedirs('finetune_data', exist_ok=True)

# Export training data
with open(f'finetune_data/train-mult.jsonl', 'w') as f:
    for example in train_examples:
        f.write(json.dumps(example) + '\n')

# Export test data
with open(f'finetune_data/test-mult.jsonl', 'w') as f:
    for example in test_examples:
        f.write(json.dumps(example) + '\n')

print(f"Exported {len(train_examples)} training examples and {len(test_examples)} test examples to JSONL files.")



Exported 70 training examples and 100 test examples to JSONL files.


In [6]:
load_dotenv()
key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=key)

In [7]:
# Initialize lists to store true labels for each variable
Res_truelabels_ls = []
Mkt_truelabels_ls = []
Aff_truelabels_ls = []
Liv_truelabels_ls = []
Adu_truelabels_ls = []
Mix_truelabels_ls = []


# Define the variables we're tracking
# variable = ['residential_add', 'market_rate_add', 'affordable_add', 'livework_add', 'adu_udu_add', 'mixeduse_add']

test_truelabels_ls = []
test_text_ls = []
for _, row in test_df.iterrows():
        # Get the text input (short_description and entitlement)
        short_desc = str(row['short_description']) if not pd.isna(row['short_description']) else ""
        entitlement = str(row['entitlement']) if not pd.isna(row['entitlement']) else ""
        
        # Combine the text inputs
        text = f"Short Description: {short_desc}\nEntitlement: {entitlement}"
        
        test_text_ls.append(text)

        # Get the label (proposed_adding)
        for var in variable:
                label = row[var]
                if pd.isna(label):
                        label = -1

                elif var == 'has_residential':
                    Res_truelabels_ls.append(label)
                elif var == 'has_market_rate':
                    Mkt_truelabels_ls.append(label)
                elif var == 'has_affordable_lowinc':
                    Aff_truelabels_ls.append(label)
                elif var == 'has_livework':
                    Liv_truelabels_ls.append(label)
                elif var == 'has_adu':
                    Adu_truelabels_ls.append(label)
                elif var == 'has_mixeduse':
                    Mix_truelabels_ls.append(label)
        

In [8]:
class Model(BaseModel):
    Res: int
    Mkt: int
    Aff: int
    Liv: int
    Adu: int
    Mix: int


Res_predlabels_ls = []
Mkt_predlabels_ls = []
Aff_predlabels_ls = []
Liv_predlabels_ls = []
Adu_predlabels_ls = []
Mix_predlabels_ls = []


for comment in test_text_ls:
  completion = client.beta.chat.completions.parse(
    model=fine_tuned_model,  # Use the fine-tuned model instead of base model
    messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": comment},
    ],
    response_format=Model,
)
  print(f'getting prediction for: {comment}')
  pred = completion.choices[0].message.parsed
  
  # print(f'predicted label: {pred.predicted_label}')
  # predlabels_ls.append(pred.predicted_label)
  Res_predlabels_ls.append(pred.Res)
  Mkt_predlabels_ls.append(pred.Mkt)
  Aff_predlabels_ls.append(pred.Aff)
  Liv_predlabels_ls.append(pred.Liv)
  Adu_predlabels_ls.append(pred.Adu)
  Mix_predlabels_ls.append(pred.Mix)

getting prediction for: Short Description: DEMOLITION OF AN EXISTING DUPLEX ACROSS TWO LOTS, AND THE CONSTRUCTION USE AND MAINTENANCE OF A SINGLE-FAMILY DWELLING ON EACH LOT
Entitlement: PURSUANT TO LAMC 12.20.2, A COASTAL DEVELOPMENT PERMIT TO ALLOW THE DEMOLITION OF AN EXISTING DUPLEX ACROSS TWO LOTS, AND THE CONSTRUCTION USE AND MAINTENANCE OF A NEW SINGLE-FAMILY DWELLING ON EACH LOT WITHIN THE SINGLE JURISDICTION OF THE COASTAL ZONE AND A MELLO ACT DETERMINATION. 

PURSUANT TO LAMC 12.28 A ZONING ADMINISTRATOR'S ADJUSTMENT TO ALLOW UP TO A 20% REDUCTION IN THE REQUIRED FRONT AND SIDE YARDS IN THE RD1.5-1-O ZONE.

AN ADMINISTRATIVE REVIEW FOR A VENICE SPECIFIC PLAN SIGN OFF.
getting prediction for: Short Description: CONVERT GARAGE TO ADU
Entitlement: CONVERT EXISTING 324 SF GARAGE TO AN ADU PER AB2299 AND SB1069.
getting prediction for: Short Description: DEMO (E) DUPLEX FOR CONSTRUCTION OF (N) 3-STORY SFD
Entitlement: PER LAMC SECTION 12.20.2 FOR A COASTAL DEVELOPMENT PERMIT, AND 

In [9]:
# Calculate accuracy for each category
categories = ['Res', 'Mkt', 'Aff', 'Liv', 'Adu', 'Mix']
true_labels_lists = [Res_truelabels_ls, Mkt_truelabels_ls, Aff_truelabels_ls, Liv_truelabels_ls, Adu_truelabels_ls, Mix_truelabels_ls]
pred_labels_lists = [Res_predlabels_ls, Mkt_predlabels_ls, Aff_predlabels_ls, Liv_predlabels_ls, Adu_predlabels_ls, Mix_predlabels_ls]

for category, true_labels, pred_labels in zip(categories, true_labels_lists, pred_labels_lists):
    correct_predictions = sum(1 for pred, true in zip(pred_labels, true_labels) if pred == true)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    
    print(f"{category} Accuracy: {accuracy:.4f} ({correct_predictions}/{total_predictions})")


Res Accuracy: 0.9400 (94/100)
Mkt Accuracy: 0.9300 (93/100)
Aff Accuracy: 0.9300 (93/100)
Liv Accuracy: 0.9900 (99/100)
Adu Accuracy: 0.9800 (98/100)
Mix Accuracy: 0.9596 (95/99)


In [10]:
# true_label_0_count = 0
# correct_predictions_for_0 = 0

# for i, true_label in enumerate(test_truelabels_ls):
#     if true_label == 0:
#         true_label_0_count += 1
#         if predlabels_ls[i] == 0:
#             correct_predictions_for_0 += 1

# accuracy_for_0 = correct_predictions_for_0 / true_label_0_count if true_label_0_count > 0 else 0
# print(f"\nAccuracy for true label = 0: {accuracy_for_0:.4f} ({correct_predictions_for_0}/{true_label_0_count})")

In [11]:
# # Print examples where the predicted label is not equal to the true label
# print("Examples where predicted label != true label:")
# count = 0
# for i, (pred_label, true_label) in enumerate(zip(predlabels_ls, test_truelabels_ls)):
#     if pred_label != true_label and count < 99:  # Limiting to 10 examples for readability
#         print(f"\nExample {i}:")
#         print(f"Text: {test_text_ls[i]}")
#         print(f"True label: {true_label}")
#         print(f"Predicted label: {pred_label}")
#         count += 1


In [12]:
# Calculate precision, recall, and F1 score for each category
categories = ['Res', 'Mkt', 'Aff', 'Liv', 'Adu', 'Mix']
true_labels_lists = [Res_truelabels_ls, Mkt_truelabels_ls, Aff_truelabels_ls, Liv_truelabels_ls, Adu_truelabels_ls, Mix_truelabels_ls]
pred_labels_lists = [Res_predlabels_ls, Mkt_predlabels_ls, Aff_predlabels_ls, Liv_predlabels_ls, Adu_predlabels_ls, Mix_predlabels_ls]

for category, true_labels, pred_labels in zip(categories, true_labels_lists, pred_labels_lists):
    # Calculate overall accuracy
    correct_predictions = sum(1 for pred, true in zip(pred_labels, true_labels) if pred == true)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    
    print(f"\n{category} Category Metrics:")
    print(f"Overall Accuracy: {accuracy:.4f} ({correct_predictions}/{total_predictions})")
    
    # Calculate metrics for class 0 and 1
    true_positives_0 = 0
    false_positives_0 = 0
    false_negatives_0 = 0
    
    true_positives_1 = 0
    false_positives_1 = 0
    false_negatives_1 = 0
    
    for pred_label, true_label in zip(pred_labels, true_labels):
        # For class 0
        if true_label == 0 and pred_label == 0:
            true_positives_0 += 1
        elif true_label != 0 and pred_label == 0:
            false_positives_0 += 1
        elif true_label == 0 and pred_label != 0:
            false_negatives_0 += 1
        
        # For class 1
        if true_label == 1 and pred_label == 1:
            true_positives_1 += 1
        elif true_label != 1 and pred_label == 1:
            false_positives_1 += 1
        elif true_label == 1 and pred_label != 1:
            false_negatives_1 += 1
    
    # Calculate metrics for class 0
    precision_0 = true_positives_0 / (true_positives_0 + false_positives_0) if (true_positives_0 + false_positives_0) > 0 else 0
    recall_0 = true_positives_0 / (true_positives_0 + false_negatives_0) if (true_positives_0 + false_negatives_0) > 0 else 0
    f1_score_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0) if (precision_0 + recall_0) > 0 else 0
    
    # Calculate metrics for class 1
    precision_1 = true_positives_1 / (true_positives_1 + false_positives_1) if (true_positives_1 + false_positives_1) > 0 else 0
    recall_1 = true_positives_1 / (true_positives_1 + false_negatives_1) if (true_positives_1 + false_negatives_1) > 0 else 0
    f1_score_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0
    
    print(f"  Class 0 Metrics:")
    print(f"    Precision: {precision_0:.4f}")
    print(f"    Recall: {recall_0:.4f}")
    print(f"    F1 Score: {f1_score_0:.4f}")
    
    print(f"  Class 1 Metrics:")
    print(f"    Precision: {precision_1:.4f}")
    print(f"    Recall: {recall_1:.4f}")
    print(f"    F1 Score: {f1_score_1:.4f}")



Res Category Metrics:
Overall Accuracy: 0.9400 (94/100)
  Class 0 Metrics:
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000
  Class 1 Metrics:
    Precision: 1.0000
    Recall: 0.9400
    F1 Score: 0.9691

Mkt Category Metrics:
Overall Accuracy: 0.9300 (93/100)
  Class 0 Metrics:
    Precision: 0.2222
    Recall: 1.0000
    F1 Score: 0.3636
  Class 1 Metrics:
    Precision: 1.0000
    Recall: 0.9286
    F1 Score: 0.9630

Aff Category Metrics:
Overall Accuracy: 0.9300 (93/100)
  Class 0 Metrics:
    Precision: 0.9176
    Recall: 1.0000
    F1 Score: 0.9571
  Class 1 Metrics:
    Precision: 1.0000
    Recall: 0.6818
    F1 Score: 0.8108

Liv Category Metrics:
Overall Accuracy: 0.9900 (99/100)
  Class 0 Metrics:
    Precision: 1.0000
    Recall: 0.9900
    F1 Score: 0.9950
  Class 1 Metrics:
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000

Adu Category Metrics:
Overall Accuracy: 0.9800 (98/100)
  Class 0 Metrics:
    Precision: 1.0000
    Recall: 0.9780
    F1 